In [30]:
import pandas as pd 
import numpy as np
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
ingr_ns=pd.read_csv('tabellen/data_ingr.csv')
ingr_schoon=pd.read_csv('tabellen/data_ingr_clean.csv') 
recipes = pd.read_csv('tabellen/Recepten_kiezen_gescheiden.csv') # Onze lijst van recepten die we wel en niet lekker vinden
recipes_wel = recipes[:80] # DataFrame met alle recpeten die wij lekker vinden
recipes_niet = recipes[80:] # DataFrame met alle recepten die wij niet lekker vinden

In [31]:
# import csv
# with open('tabellen/Recepten_kiezen_f.csv', newline='') as csvfile:
#     csvlezer = csv.reader(csvfile, delimiter=',', quotechar='"')
#     for x in csvlezer:
#         x = recipes
# recipes[117:123]

# recipes

In [32]:
onze_ns = ingr_ns[ingr_ns['recipe'].isin(recipes['title'])] # DataFrame van de ongeschoonde data met alle ingredienten van onze gekozen gerechten
onze_schoon = ingr_schoon[ingr_schoon['recipe'].isin(recipes['title'])] # DataFrame van de opgeschoonde data met alle ingredienten van onze gekozen gerechten


In [33]:
class Recepten: 
    
    def __init__(self, data_ingr, schoongemaakt=False): 
        # Initialisatie van de klasse met het pad naar de data en of de data schoon is 
        self.data_ingr = data_ingr
        self.schoongemaakt = schoongemaakt
        
    def laad_ingr(self):
        # Methode die de ingrediëntentabel inlaadt
        self.tabel = pd.read_csv(self.data_ingr)
        
    def aanbeveling(self, fav_recepten):
        # Alle favoriete ingredienten worden gezocht, dmv set worden de duplicates weggehaald
        self.fav_ingr = list(set(self.tabel['ingredient'].loc[self.tabel['recipe'].isin(fav_recepten)]))
        # Initialisatie van de CountVectorizer die text data convergeert naar getallen
        count_vectorizer = CountVectorizer()
        # Convergeert fav_ingr naar een matrix van tokens
        self.fav_matrix = count_vectorizer.fit_transform(self.fav_ingr)
        # Initialiseer een lege lijst om alle recepten op te slaan
        titel_lijst = []
        # Itereer door alle recepten, maar filter de favoriete eruit en voorkom ook dubbele receptnamen
        [titel_lijst.append(recept) for recept in self.tabel['recipe'] if recept not in fav_recepten and recept not in titel_lijst]
        # Initialiseer een lege lijst om de gelijkenisscore in te bewaren tussen de fav recepten en alle andere
        gelijkenis = []
        # Bereken de cosinusgelijkenis tussen de fav recepten en alle ander
        for recept in titel_lijst:
            # Kies de ingredientelijst van het huidige recept
            recept_lijst = list(self.tabel['ingredient'].loc[self.tabel['recipe']==recept])
            # Convergeer de ingredientenlijst naar een matrix van token obv de fav_matrix
            rec_matrix = count_vectorizer.transform(recept_lijst)
            # Bereken de cosinusgelijkenis tussen het huidige recept en de favoriete ingredienten, 
            # dmv van een vector bestaand uit enen, evengroot als het aantal ingredienten in het huidige recept
            cosinus = cosine_similarity(np.ones(np.size(rec_matrix.toarray())).reshape(1,-1), rec_matrix.reshape(1,-1))
            # Voeg de gelijkenis score en het recept toe aan de lijst
            gelijkenis.append([cosinus[0][0],recept])
        
        # Sorteer de lijst aan gelijkenissen op aflopende volgorde
        gelijkenis.sort(reverse=True)
        # Selecteer de top 10 aanbevolen recepten op basis van de cosinusgelijkenis
        self.aanbeveling = [titel[1] for titel in gelijkenis][:10]
        
        return self.aanbeveling
        
recepten_klasse = Recepten('tabellen/data_ingr_clean.csv',schoongemaakt=True)
recepten_klasse.laad_ingr()

# fav_recepten = recipes_wel['title'].tolist()
# aanbevolen = recepten_klasse.aanbeveling(fav_recepten)
# print('De volgende 10 recepten worden aanbevolen: ', aanbevolen)

In [35]:
recepten_kika = recipes_wel[:20]
recepten_araksan = recipes_wel[20:40]
recepten_fenne = recipes_wel[40:60]
recepten_phea = recipes_wel[60:80]

recepten_klasse_k = Recepten('tabellen/data_ingr_clean.csv',schoongemaakt=True)
recepten_klasse_k.laad_ingr()
fav_recepten_k = recepten_kika['title'].tolist()
aanbevolen_k = recepten_klasse_k.aanbeveling(fav_recepten_k)
print('De volgende 10 recepten worden aanbevolen voor Kika: ', aanbevolen_k)
# precision@10: 8/10= 80%

De volgende 10 recepten worden aanbevolen voor Kika:  ['Snelle Thaise viskoekjes met chilisaus', "Volkorenpincho's", 'Asperges koken', 'Snelle noedelroerbak met groene currypasta en een eitje', 'Mexicaanse maaltijdsoep van zoete aardappel met kidneybonen ', 'Yoghurtmarinade met baharat', 'Zoete aardappel uit de airfryer', 'Kruidendip', 'Kaasvlinders', 'Pittige zoete-aardappelspread']


In [36]:
recepten_klasse_a = Recepten('tabellen/data_ingr_clean.csv',schoongemaakt=True)
recepten_klasse_a.laad_ingr()
fav_recepten_a = recepten_araksan['title'].tolist()
aanbevolen_a = recepten_klasse_a.aanbeveling(fav_recepten_a)
print('De volgende 10 recepten worden aanbevolen voor Araksan: ', aanbevolen_a)
# precision@10: 8/10= 80%

De volgende 10 recepten worden aanbevolen voor Araksan:  ['Kerstballetjes met gesmolten kaas', "Burrito'-schotel met knapperige tortillapuntjes", 'Mexicaanse maaltijdsoep van zoete aardappel met kidneybonen ', 'Frisse yoghurtsaus met gegrilde paprika', 'Gepofte aardappel met tonijn', 'Traybake met kikkererwten, kipgehaktballetjes en garam masala', 'Aardappel-tonijnsalade met asperges', 'Homemade tomatensaus', 'Pastasalade met mais &amp; paprika', 'Mexicaanse tonijnwraps']


In [39]:
recepten_klasse_f = Recepten('tabellen/data_ingr_clean.csv',schoongemaakt=True)
recepten_klasse_f.laad_ingr()
fav_recepten_f = recepten_fenne['title'].tolist()
aanbevolen_f = recepten_klasse_f.aanbeveling(fav_recepten_f)
print('De volgende 10 recepten worden aanbevolen voor Fenne: ', aanbevolen_f)
# precision@10: 7/10= 70%

De volgende 10 recepten worden aanbevolen voor Fenne:  ["Volkorenpincho's", "Burrito'-schotel met knapperige tortillapuntjes", 'Kruidendip', 'Farro met paprika-tomatensaus', 'Italiaanse couscous ', 'Fusilli met groente en ham-kaassaus', 'Zelfgemaakt pistache-ijs', 'Omelet bakken', 'Frisse yoghurtsaus met gegrilde paprika', 'Tomatensaus']


In [38]:
recepten_klasse_p = Recepten('tabellen/data_ingr_clean.csv',schoongemaakt=True)
recepten_klasse_p.laad_ingr()
fav_recepten_p = recepten_phea['title'].tolist()
aanbevolen_p = recepten_klasse_p.aanbeveling(fav_recepten_p)
print('De volgende 10 recepten worden aanbevolen voor Phea: ', aanbevolen_p)
# precision@10: 8/10= 80%

De volgende 10 recepten worden aanbevolen voor Phea:  ['Snelle Thaise viskoekjes met chilisaus', "Marokkaanse' lamsbout", 'Banaan met chocolade en rum', 'Carpaccio-wraps', 'Noedelroerbak met garnalen, rode curry &amp; broccoli', 'Hemelse modder', 'Meringuetaart met praliné en salted caramel', 'Zelfgemaakte chocolademelk', 'Zelfgemaakt pistache-ijs', 'Kruidendip']


## Niet schoongemaakt

In [40]:
recepten_klasse_k = Recepten('tabellen/data_ingr.csv',schoongemaakt=False)
recepten_klasse_k.laad_ingr()
fav_recepten_k = recepten_kika['title'].tolist()
aanbevolen_k = recepten_klasse_k.aanbeveling(fav_recepten_k)
print('De volgende 10 recepten worden aanbevolen voor Kika: ', aanbevolen_k)
# precision@10: 7/10= 70%

De volgende 10 recepten worden aanbevolen voor Kika:  ['Yoghurtmarinade met baharat', 'Zoete aardappel uit de airfryer', 'Asperges koken', 'Olijf-artisjokspread met ansjovis', 'Snelle noedelroerbak met groene currypasta en een eitje', 'Mexicaanse maaltijdsoep van zoete aardappel met kidneybonen ', 'Kruidendip', 'Crostini met gerookte makreel', 'Basisrecept vegan meringue', 'Bloemkoolrijst-schotel met rode kidneybonen']


In [41]:
recepten_klasse_a = Recepten('tabellen/data_ingr.csv',schoongemaakt=False)
recepten_klasse_a.laad_ingr()
fav_recepten_a = recepten_araksan['title'].tolist()
aanbevolen_a = recepten_klasse_a.aanbeveling(fav_recepten_a)
print('De volgende 10 recepten worden aanbevolen voor Araksan: ', aanbevolen_a)
# precision@10: 6/10= 60%

De volgende 10 recepten worden aanbevolen voor Araksan:  ['Frisse yoghurtsaus met gegrilde paprika', 'Mexicaanse maaltijdsoep van zoete aardappel met kidneybonen ', 'Olijf-artisjokspread met ansjovis', 'Tortillataart', 'Gepofte aardappel met tonijn', 'Aardappel-tonijnsalade met asperges', 'Appelringpannenkoeken', 'Traybake met kikkererwten, kipgehaktballetjes en garam masala', 'Tomatensaus', 'Hashbrown met paprika en eieren']


In [42]:
recepten_klasse_f = Recepten('tabellen/data_ingr.csv',schoongemaakt=False)
recepten_klasse_f.laad_ingr()
fav_recepten_f = recepten_fenne['title'].tolist()
aanbevolen_f = recepten_klasse_f.aanbeveling(fav_recepten_f)
print('De volgende 10 recepten worden aanbevolen voor Fenne: ', aanbevolen_f)
# precision@10: 7/10= 70%

De volgende 10 recepten worden aanbevolen voor Fenne:  ['Crostini met gerookte makreel', 'Courgette bruschetta koolhydraatarm', 'Olijf-artisjokspread met ansjovis', 'Kruidendip', 'Frisse yoghurtsaus met gegrilde paprika', 'Farro met paprika-tomatensaus', 'Tortelloni met verse kruiden en Parmezaan', 'Hashbrown met paprika en eieren', 'Hartige taart met broccoli en bloemkool', 'Glutenvrije pannenkoeken']


In [43]:
recepten_klasse_p = Recepten('tabellen/data_ingr.csv',schoongemaakt=False)
recepten_klasse_p.laad_ingr()
fav_recepten_p = recepten_phea['title'].tolist()
aanbevolen_p = recepten_klasse_p.aanbeveling(fav_recepten_p)
print('De volgende 10 recepten worden aanbevolen voor Phea: ', aanbevolen_p)
# precision@10: 6/10= 60%

De volgende 10 recepten worden aanbevolen voor Phea:  ['Carpaccio-wraps', 'Noedelroerbak met garnalen, rode curry &amp; broccoli', 'Hemelse modder', 'Zalm met kruidenkorst', 'Lauwwarme salade van gegrilde zoete aardappel', 'Zoete-aardappelstamppot met prei, geitenkaas en walnoten', 'Vegan pasta pesto', 'Tabouleh', 'Gezonde eiwraps met spinazie, biet en geitenkaas', 'Zelfgemaakt pistache-ijs']


In [39]:
# recept_schoon = Recepten(ingr_schoon)

# print(recept_schoon.data_frame)

AttributeError: 'Recepten' object has no attribute 'data_frame'

In [14]:
# recept_ns = Recepten(ingr_ns)

# print(recept_ns.data_frame)

       Unnamed: 0                            recipe            ingredient  \
0               0     Kruidnoten met choco-discodip         melkchocolade   
1               1     Kruidnoten met choco-discodip            kruidnoten   
2               2     Kruidnoten met choco-discodip              discodip   
3               3           Kruidnoten in marsepein      blanke marsepein   
4               4           Kruidnoten in marsepein  ongezouten roomboter   
...           ...                               ...                   ...   
49814       71804  Biefstuk met rodewijnsaus en ham             olijfolie   
49815       71805  Biefstuk met rodewijnsaus en ham           biefstukken   
49816       71806  Biefstuk met rodewijnsaus en ham                 boter   
49817       71807  Biefstuk met rodewijnsaus en ham            serranoham   
49818       71808  Biefstuk met rodewijnsaus en ham             rode wijn   

       quantity     unit  
0         100.0        g  
1         100.0      

In [15]:
recipes_schoon = recipes.dropna(subset=['title'])
recipes_schoon

,Unnamed: 0,title,persons,time,calories,stars,url,image
0,Courgette carbonara,4,20,575,0,https://www.ah.nl/allerhande/recept/R-R1188954...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
0,Mexicaanse taco's met vis en zoete aardappel,4,20,655,0,https://www.ah.nl/allerhande/recept/R-R1188828...,https://static.ah.nl/static/recepten/img_09292...,NaN
0,Gegrilde-paprikasalade met feta en munt,4,15,205,0,https://www.ah.nl/allerhande/recept/R-R1188715...,https://static.ah.nl/static/recepten/img_09094...,NaN
0,Viscurry met zelfgemaakte currypasta,4,20,345,0,https://www.ah.nl/allerhande/recept/R-R1187967...,https://static.ah.nl/static/recepten/img_08598...,NaN
0,Spruitjesgratin met zoete aardappel,4,10,665,0,https://www.ah.nl/allerhande/recept/R-R1187741...,https://static.ah.nl/static/recepten/img_08516...,NaN
...,...,...,...,...,...,...,...,...
0,Rodekoolsteak met blauwaderkaas en truffel,4,15,550,0,https://www.ah.nl/allerhande/recept/R-R1189726...,https://static.ah.nl/static/recepten/img_10034...,NaN
0,Vegan wraps met tofu en rodekoolsalade,4,20,670,0,https://www.ah.nl/allerhande/recept/R-R1195491...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
0,Hachee uit de oven,4,15,365,0,https://www.ah.nl/allerhande/recept/R-R1193014...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
0,Aardappel en zuurkool,4,15,405,0,https://www.ah.nl/allerhande/recept/R-R758540/...,https://static.ah.nl/static/recepten/img_00767...,NaN


In [16]:
a= np.array([recipes_schoon['title']]) 
a 

array([[ 4,  4,  4,  4,  4,  4,  4,  4,  4, 12,  4,  4,  4,  4,  1,  4,
        20, 24, 20,  4,  4,  4,  4,  4,  4, 10, 16,  4,  4,  2,  4, 12,
         4,  8,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4, 12,  4, 20,  4, 12, 12,  4,  4,  4,
         4,  4,  4,  8,  4,  6, 12,  4,  4,  8,  4,  4,  4,  2,  4,  4,
         4,  2,  4,  2,  4,  4,  4,  1,  2,  4,  4,  4,  4,  4,  4,  4,
         8,  4, 12,  4,  4, 20,  2,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4]])

In [32]:
set_ingr= { ingr_schoon['ingredient'][i] for i in range(0, len(ingr_schoon)) if ingr_schoon['recipe'][i] in a}
set_ingr 

{'100% notenpasta amandel',
 'AH BASIC spaghetti',
 'AH Biologisch carpaccio met kaas, dressing en pijnboompitten',
 'Aardappel',
 'BYB red cheddar',
 'Balsajo zwarte knoflook',
 'Chioggia biet',
 'Country Cow Cheese slices',
 'De Zaanse Hoeve ongezouten roomboter',
 'De Zaanse Hoeve verse slagroom',
 'Euroma Original Spices Thai green curry',
 'Gelderse rookworst',
 'Go-Tan volkorennoedels',
 'Goudse belegen komijnekaas 48+',
 'Grand’Italia Sugocasa Erbe in fles',
 'Grand’Italia gnocchi',
 'HAK linzen in zak',
 'Hollandse bruine bonen uit pot',
 'Italiaanse roerbakgroente champignons',
 'Kaas',
 'Kipfilet',
 'Kool',
 'Kruidenmix tex mex',
 'Kühne mierikswortel in pot',
 'Peen',
 'Zalm',
 'ahornsiroop',
 'amandeldrink',
 'amandelen dry roasted',
 'amandelmeel',
 'amandelschaafsel',
 'amandelspijs',
 'ananas',
 'ansjovisfilets in olie in blik',
 'appel',
 'appelciderazijn',
 'appelmoes 0%',
 'arachideolie',
 'aspergebroccoli',
 'aubergine',
 'augurkenblokjes',
 'azijn',
 'babyromainesla